In [1]:
# https://codepen.io/barney-parker/pen/QjXOmv/
# https://freefrontend.com/css-timelines/

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("input/sheet.xlsx", sheet_name='schedule')
data['Day'] = data["Day"].fillna(method='ffill')
data.head(10)

,Day,Unnamed: 1,Start,End,DurationStr,FirstName,LastName,Title,TalkType,DurA,DurM,Unnamed: 11,Chair,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Type,Dur. (A),Dur. (M),Type,Chair,NaN,NaN,NaN
1,Monday,Morning,09:15:00,09:30:00,NaN,Welcome,,NaN,Organizers,0,15,NaN,NaN,NaN,NaN,NaN
2,Monday,NaN,09:30:00,10:30:00,(45+15),Sophia,Economou,Efficient variational quantum algorithms,Spotlight,60,NaN,QC,Filippo Vicentini,NaN,NaN,NaN
3,Monday,NaN,10:30:00,11:05:00,(25+10),Patrick,Hümbeli,Entanglement Forging with generative neural ne...,Regular,35,NaN,QC,NaN,NaN,NaN,NaN
4,Monday,NaN,11:05:00,11:35:00,NaN,Break,NaN,NaN,break,0,30,NaN,NaN,NaN,NaN,NaN
5,Monday,NaN,11:35:00,12:10:00,(25+10),George,Booth,A Bayesian machine-learning approach to the qu...,Regular,35,NaN,CC,NaN,NaN,NaN,NaN
6,Monday,NaN,12:10:00,12:45:00,(25+10),Markus,Schmitt,Neural quantum states to simulate non-equilibr...,Regular,35,NaN,CC,NaN,NaN,NaN,NaN
7,Monday,NaN,12:45:00,14:00:00,NaN,Lunch,NaN,NaN,break,0,75,NaN,NaN,NaN,NaN,NaN
8,Monday,Afternoon,14:00:00,14:35:00,(25+10),Yusuke,Nomura,Artificial neural networks for exploring quant...,Regular,35,NaN,CC,Tess Smidt,NaN,NaN,NaN
9,Monday,NaN,14:35:00,15:10:00,(25+10),Attila,Szabó,Symmetries in neural quantum states: group-con...,Regular,35,NaN,CC,NaN,NaN,NaN,NaN


In [3]:
# def make_talk_html(start_time, end_time, first_name, last_name, title, abstract=None):
#     print(start_time)
#     block = """
#     <li class="event" data-date="{start_time} - {end_time}">
#       <div class="member-infos">
#         <h1 class='member-title'>{first_name} {last_name}</h1>
#         <h2 class="member-location ">Title: {title}</h2>
#       </div>
#     </li>
#     """.format(
#         start_time=start_time.strftime('%H:%M'),
#         end_time=end_time.strftime('%H:%M'),
#         title=title,
#         first_name=first_name,
#         last_name=last_name
#     )
#     return block

# def make_other_html(start_time, end_time, title):
#     block = """
#     <li class="event" data-date="{start_time} - {end_time}">
#       <div class="member-infos">
#         <h1 class='member-title'>{title}</h1>
#       </div>
#     </li>
#     """.format(
#         start_time=start_time.strftime('%H:%M'),
#         end_time=end_time.strftime('%H:%M'),
#         title=title,
#     )
#     return block

In [4]:
def make_talk_html(start_time, end_time, first_name, last_name, title, talk_type):
#     print(start_time)
    delta = {
        "Spotlight" : "45+15",
        "Regular"   : "25+10",
        "Short"     : "15+5"
    }[talk_type]
#     block = (
#         f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')} ({delta})\n" +
#         f"    \n" +
#         f"        {first_name} {last_name}: {title}\n\n\n"
#     )
    name = f"{first_name} {last_name}"
    block = (
        f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')} ({delta:5})"+
        f"\n{name:19}: {title}\n\n"
    )
    return block

def make_other_html(start_time, end_time, title):
    block = (
        f"{start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}: {title.upper()}\n\n"
    )
    return block

In [5]:
from collections import defaultdict
html_blocks = defaultdict(list)

for day, df in data.groupby("Day"):
    
    df["Start"] = pd.to_datetime(df["Start"].astype(str).apply(pd.Timestamp))
    df["End"] = pd.to_datetime(df["End"].astype(str).apply(pd.Timestamp))
    df = df.sort_values(by="Start")
    
    for _, row in df.iterrows():
        #print(row)
        start_time = row.Start
        end_time = row.End
        day = row.Day
        #print(row.Name)  
        if not isinstance(row.Type, str):
            #print("Skip row")
            continue
        
        if row.Type in ('Spotlight', 'Regular', 'Short'):
#             print("Create one")
            first_name = row.Name
            last_name = row.Surname
            title = "TBD"
            code = make_talk_html(start_time, end_time, first_name, last_name, title, row.Type)
            html_blocks[day].append(code)

        else:
            title = row.Type
            code = make_other_html(start_time, end_time, title)
            html_blocks[day].append(code)
            
html_blocks = dict(html_blocks)
print(list(html_blocks.keys()));

AttributeError: 'Series' object has no attribute 'Type'

In [ ]:
html_blocks["Monday"][-1]

In [6]:
# with open("html_header.html", 'r') as f:
#     html_header = f.readlines()
#     html_header = "".join(html_header)
# with open("html_footer.html", 'r') as f:
#     html_footer = f.readlines()
#     html_footer = "".join(html_footer)
# with open("css_header.txt", 'r') as f:
#     css_header = f.readlines()
#     css_header = ["<style type=\"text/css\">\n"] + css_header + ["\n</style>\n"] 
#     css_header = css_header 
#     css_header = "".join(css_header)
    
# html_code = "".join(html_blocks["Monday"])
# # html_code = html_blocks["Monday"][1]
# html_code

In [11]:
days = ("Monday", "Tuesday", "Wednesday", "Thursday", "Friday")

html_code = ""
for day in days:
    v = html_blocks[day]
    html_code += f"\n{day.upper()}\n\n"
    html_code += "".join(v)
# html_code = html_blocks["Monday"][1]
html_code

"\nMONDAY\n\n09:15 - 09:30: ORGANIZERS\n\n09:30 - 10:30 (45+15)\nSophia Economou    : TBD\n\n10:30 - 11:05 (25+10)\nPatrick Huembeli   : TBD\n\n11:05 - 11:35: BREAK\n\n11:35 - 12:10 (25+10)\nGeorge Booth       : TBD\n\n12:10 - 12:45 (25+10)\nOlivier Gauthé     : TBD\n\n12:45 - 14:15: BREAK\n\n14:15 - 15:15 (45+15)\nMax Welling        : TBD\n\n15:15 - 15:50 (25+10)\nAtilla Szabó       : TBD\n\n15:50 - 16:20: BREAK\n\n16:20 - 16:40 (15+5 )\nGiulia Mazzola     : TBD\n\n16:40 - 17:15 (25+10)\nYusuke Nomura      : TBD\n\n\nTUESDAY\n\n09:15 - 09:50 (25+10)\nMartin Kliesch     : TBD\n\n09:50 - 10:25 (25+10)\nMarkus  Schmitt    : TBD\n\n10:25 - 10:55: BREAK\n\n10:55 - 11:30 (25+10)\nChristian Kokail   : TBD\n\n11:30 - 12:05 (25+10)\nEnrico Rinaldi     : TBD\n\n12:05 - 13:25: BREAK\n\n13:25 - 14:25 (45+15)\nTess Schmidt       : TBD\n\n14:25 - 14:45 (15+5 )\nPaolo Stornati     : TBD\n\n14:45 - 15:00: BREAK\n\n15:00 - 15:35 (25+10)\nStephen Clark      : TBD\n\n15:35 - 17:35: UNCONFERENCE\n\n\nWED

In [12]:
with open("output.html", 'w') as f:
#     code = (css_header, html_header, html_code, html_footer)
    code = (html_code,)
    code = "\n".join(code)
    f.write(code)